<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/pytorch-computer%20vision/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모델 학습

torchinfo 라이브러리로 모델 구조 확인

모델과 입력데이터를 전달했을 때 각 계층의 출력 형태와 매개변수를 확인할 수 있다.

모델에 입력하는 텐서 데이터의 형태에 따라 출력 형태가 바뀐다

In [2]:
pip install torchinfo

In [3]:
from torchvision import models
from torchinfo import summary


model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1")
# 사전학습된 알렉스넷 모델 가중치 불러오기
# None 으로 설정하면 사전학습된 가중치 불러오지 않음
summary(model, (1, 3, 224, 224), device="cpu")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 96.2MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

출력 결과는 알렉스넷에 (1, 3, 224, 224) 크기의 텐서를 입력했을 때 계층마다 출력하는 특징 맵의 크기와 필요한 매개변수의 수를 보여준다.

이미지넷으로 사전 학습된 알렉스넷 모델이다. 이미지넷은 1,000 개의 클래스로 구성된 데이터세트이므로 사전 학습된 알렉스넷은 1,000개의 클래스에 대한 예측을 수행

#### 클래스 정보 불러오기

In [4]:
with open("imagenet_classes.txt", 'r') as file:
  classes = file.read().splitlines()

print(f"클래스 개수 : {len(classes)}")
print(f"첫 번째 클래스 레이블 : {classes[0]}")

클래스 개수 : 1000
첫 번째 클래스 레이블 : tench


#### 이미지 데이터 전처리

In [5]:
import torch
from PIL import Image
from torchvision import models, transforms

transform = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ) # 이미지는 3차원이므로 3개(이미지넷 데이터세트에서 사용된 이미지들 평균과 표준편차 값)
    ]
)

device = "cuda" if torch.cuda.is_available() else 'cpu'
model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1").eval().to(device)

tensors = []
files = ['bus.jpg', 'airplane.jpg']

for file in files:
  image = Image.open(file)
  tensors.append(transform(image))

tensors = torch.stack(tensors)
print(f"입력 텐서의 크기: {tensors.shape}")

입력 텐서의 크기: torch.Size([2, 3, 224, 224])


## 모델 추론

In [7]:
import numpy as np
from torch.nn import functional as F

with torch.no_grad():
  outputs = model(tensors.to(device)) # 순전파 수행, 출력값은 입력텐서의 배치 크기와 동일([배치크기, 클래스 개수])
  print(outputs)
  probs = F.softmax(outputs, dim=-1) # 소프트맥스 함수로 값을 활성화
  print(probs)
  top_probs, top_idxs = probs.topk(5) # topk 메서드로 텐서 값이 가장 큰 상위 다섯개 요소 반환

# 확률, 인덱스, 클래스를 넘파이 배열로 변환
top_probs = top_probs.detach().cpu().numpy()
top_idxs = top_idxs.detach().cpu().numpy()
top_classes = np.array(classes)[top_idxs]

for idx, (cls, prob) in enumerate(zip(top_classes, top_probs)):
    print(f"{files[idx]} 추론 결과")
    for c, p in zip(cls, prob):
        print(f" - {c:<30} : {p * 100:>5.2f}%")

tensor([[-4.0288, -0.1984,  2.1301,  ..., -5.3826, -2.6092,  1.2260],
        [-0.0914, -4.3196,  2.9599,  ..., -1.7398,  4.6348, -1.3781]])
tensor([[3.4603e-13, 1.5946e-11, 1.6365e-10,  ..., 8.9371e-14, 1.4311e-12,
         6.6264e-11],
        [7.5438e-10, 1.0998e-11, 1.5949e-08,  ..., 1.4511e-10, 8.5144e-08,
         2.0836e-10]])
bus.jpg 추론 결과
 - streetcar                      : 60.25%
 - trolleybus                     : 37.99%
 - minibus                        :  1.54%
 - passenger car                  :  0.17%
 - recreational vehicle           :  0.03%
airplane.jpg 추론 결과
 - airliner                       : 66.83%
 - warplane                       : 20.12%
 - wing                           :  9.29%
 - space shuttle                  :  2.89%
 - missile                        :  0.38%


결과를 보면 bus는 streetcar 나 trolleybus로 예측

비행기는 airliner이나 warplane로 유사한 클래스가 가장 높게 예측된다.